In [2]:
import os
import azure.cognitiveservices.speech as speechsdk

# configuration for Azure Speech
speech_config = speechsdk.SpeechConfig(subscription=os.environ.get('SPEECH_KEY'), region=os.environ.get('SPEECH_REGION'))
# The language of the voice that speaks for TTS
speech_config.speech_synthesis_voice_name='en-US-JennyNeural'
# The language of text for SST
speech_config.speech_recognition_language="en-US"
# Audio configuration for TTS
audio_config_tts = speechsdk.audio.AudioOutputConfig(use_default_speaker=True)
# Audio configration for SST
audio_config_sst = speechsdk.AudioConfig(use_default_microphone=True)

# create speech synthesizer for TTS
speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=audio_config_tts)
# create speech recognizer for SST
speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config_sst)

## Sample code for text to speech

In [2]:
def text_to_speech(text):
    speech_synthesis_result = speech_synthesizer.speak_text_async(text).get()
    if speech_synthesis_result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
        print("Speech synthesized for text [{}]".format(text))
    elif speech_synthesis_result.reason == speechsdk.ResultReason.Canceled:
        cancellation_details = speech_synthesis_result.cancellation_details
        print("Speech synthesis canceled: {}".format(cancellation_details.reason))
        if cancellation_details.reason == speechsdk.CancellationReason.Error:
            if cancellation_details.error_details:
                print("Error details: {}".format(cancellation_details.error_details))
                print("Did you set the speech resource key and region values?")

## Sample code for speech to text

In [3]:
def recognize_from_microphone():

    print("Speak into your microphone.")
    speech_recognition_result = speech_recognizer.recognize_once_async().get()

    if speech_recognition_result.reason == speechsdk.ResultReason.RecognizedSpeech:
        print("Recognized: {}".format(speech_recognition_result.text))
    elif speech_recognition_result.reason == speechsdk.ResultReason.NoMatch:
        print("No speech could be recognized: {}".format(speech_recognition_result.no_match_details))
    elif speech_recognition_result.reason == speechsdk.ResultReason.Canceled:
        cancellation_details = speech_recognition_result.cancellation_details
        print("Speech Recognition canceled: {}".format(cancellation_details.reason))
        if cancellation_details.reason == speechsdk.CancellationReason.Error:
            print("Error details: {}".format(cancellation_details.error_details))
            print("Did you set the speech resource key and region values?")

#recognize_from_microphone()

### Testing input code for SST

In [4]:
# Get text from the console and synthesize to the default speaker.
print("Enter some text that you want to speak >")
text = input()
print(type(text))
text_to_speech(text)

Enter some text that you want to speak >


 Hello


<class 'str'>
Speech synthesized for text [Hello]


### Combine LLM with Azure Speech Service

In [3]:
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ["OPENAI_API_BASE"] = 'https://qucy-openai-test.openai.azure.com/'
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"

# create LLM
llm = AzureChatOpenAI(deployment_name="chatGPTAzure", model_name="gpt-35-turbo")

conversation = ConversationChain(
    llm=llm,
    memory=ConversationBufferWindowMemory(k=2),
    verbose=True
)

In [4]:
response = conversation.predict(input='What is Machine Learning ?')
response
#text_to_speech(response)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: What is Machine Learning ?
AI:

> Finished chain.


'Machine Learning is a subfield of Artificial Intelligence that involves the use of algorithms and statistical models to enable computer systems to learn from data, without being explicitly programmed. It involves the development of algorithms that can automatically identify patterns in data and use those patterns to make predictions or decisions. Machine Learning has applications in a wide range of fields, including computer vision, natural language processing, robotics, and finance, among others. Would you like me to provide more information or examples?'

In [5]:
from langchain.llms import AzureOpenAI

llm_1 = AzureOpenAI(deployment_name="text-davinci-003", model_name="text-davinci-003")

In [6]:
llm_1('Hello')

'\n\nWelcome to the site. I hope you have a great time here. If you need any help, please let me know.'

In [7]:
llm_2 = AzureOpenAI(temperature=0, deployment_name="chatGPTAzure", model_name="gpt-35-turbo")
llm_2('Hello')

', I am a professional writer with over 5 years of experience writing for clients all over the world. I can research and write top notch articles and content with 100% uniqueness because I have a copyscape premium More\n\nHello, I am a professional content writer with SEO Expertise. I can provide you with Unique and Creative Articles/Blogs/Content/Research paper and many more on given Niche/Theme at Time Deadlines. No Gramm More\n\nHello, I am a professional content writer with SEO Expertise. I can provide you with Unique and Creative Articles/Blogs/Content/Research paper and many more on given Niche/Theme at Time Deadlines. No Gramm More\n\nHello, I am a professional content writer with SEO Expertise. I can provide you with Unique and Creative Articles/Blogs/Content/Research paper and many more on given Niche/Theme at Time Deadlines. No Grammatical More\n\nHello, I am a professional content writer with SEO Expertise. I can provide you with Unique and Creative Articles/Blogs/Content/Re